In [84]:
import numpy as np
import pandas as pd
import datetime
import random
import matplotlib.pyplot as plt

In [136]:
def random_date(start, end):
    """Generate a random datetime between `start` and `end`"""
    return start + datetime.timedelta(
        # Get a random amount of seconds between `start` and `end`
        seconds=random.randint(0, int((end - start).total_seconds())),
    )

In [137]:
def to_date(start, end, r):
    """Generate a random datetime between `start` and `end`"""
    return start + datetime.timedelta(
        # Get a random amount of seconds between `start` and `end`
        seconds=int(r*int((end - start).total_seconds())),
    )

# Niniejszy dokument poświęcony jest przygotowaniu zbioru danych: Magazyn

Opis zbioru:
Kolumny:
+ pk: warehouse_record_id
+ unikalny_kod_encji: entity_id
+ typ_produktu: product_id
+ ilość: quantity
+ cena: price
+ data_wazności: expiration_date
+ data_od: from_date
+ data_do: to_date
+ aktywność wpisu: activ

Dlaczego taka tabela ma sens: w dużym magazynie za każdym razem, gdy coś wchodzi do magazynu lub jest z niego usuwane lub updatowane 

In [138]:
def to_day(y):
    return datetime.datetime(y.year, y.month, y.day)

In [139]:
def sample(f, n):
    R = []
    i = 0
    while i < n:
        r = np.random.rand()
        if np.random.rand() < f(r):
            R.append(r)
            i += 1
    return np.array(R)

In [140]:
datetime.timedelta(days=5)

datetime.timedelta(days=5)

In [187]:
def generate_magazine(start, end, general_trend, prod,
                      freq, quant, std_quant, exp_time, price, n):
    
    frequency = freq/freq.sum()
    
    products = np.random.choice(prod, size=n, p=frequency)
    
    quantity = np.int_(np.array([std_quant[p] for p in products]) * np.random.rand(n) + np.array([quant[p] for p in products]))
    
    R = sample(general_trend, n)
    time_in = [to_day(to_date(start, end, r)) for r in R]
    
    exp_date = np.array(time_in) - datetime.timedelta(days=3) + np.array([exp_time[products[i]] for i in range(n)])
    
    time_out = [to_day(random_date(time_in[i], time_in[i] + exp_time[products[i]])) for i in range(n)]
    
    is_in = [end < t for t in time_out]
    
    time_out = [t if not end < t else None for t in time_out]
    
    price = [price[p] for p in products]
    
    return pd.DataFrame({'product_id': products,
                        'quantity':quantity,
                        'price':price,
                        'expiration_date':exp_date,
                        'time_in':time_in,
                        'time_out':time_out,
                        'active':is_in})

In [188]:
df = generate_magazine(datetime.datetime(2019,6,1),
                  datetime.datetime(2020,6,1),
                 lambda x: np.sin(300*x)/4 + .75,
                  np.array([1,2,3]),
                  np.array([10, 1, 3]),
                    {1:50, 2:1000, 3:10},
                    {1:50, 2:100,3:10},
                    {1: datetime.timedelta(days=5),
                    2: datetime.timedelta(days=5),
                    3: datetime.timedelta(days=5)},
                 {1:10,2:20,3:30}, 100000)
                    
                    

In [193]:
df[df.expiration_date == df.time_out]

,product_id,quantity,price,expiration_date,time_in,time_out,active
2,1,96,10,2020-02-27,2020-02-25,2020-02-27,False
4,1,70,10,2020-05-27,2020-05-25,2020-05-27,False
5,1,69,10,2019-06-26,2019-06-24,2019-06-26,False
12,3,15,30,2019-12-12,2019-12-10,2019-12-12,False
20,1,99,10,2019-11-28,2019-11-26,2019-11-28,False
...,...,...,...,...,...,...,...
99971,3,16,30,2019-12-27,2019-12-25,2019-12-27,False
99987,1,82,10,2020-04-20,2020-04-18,2020-04-20,False
99988,1,95,10,2019-06-19,2019-06-17,2019-06-19,False
99989,3,10,30,2020-02-19,2020-02-17,2020-02-19,False
